In [2]:
import pandas as pd

In [4]:
all_holidays_victoria_df = pd.read_csv("all_holidays_victoria.csv")

In [ ]:
all_holidays_victoria_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2851 entries, 0 to 2850
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dateType        2851 non-null   object
 1   name            2851 non-null   object
 2   important_date  2851 non-null   object
 3   publisher       2851 non-null   object
 4   description     1680 non-null   object
 5   source          2795 non-null   object
dtypes: object(6)
memory usage: 133.8+ KB


In [ ]:
all_holidays_victoria_df

In [9]:
all_holidays_victoria_df["name"].unique()

array(['Lent', 'Great lent (Orthodox)', 'Purim', 'Ridvan', 'Pesach',
       'Ramadan', 'Krishna Janmashtami', 'Declaration of the Bab',
       "Tisha B'Av", 'Navaratri', 'Sukkot', 'Advent', 'All Saints day',
       'Hanukkah', 'Maghi', 'Lailat al Miraj', 'Holy Week',
       'Great Lent (Orthodox)', 'Eid Al Adha', 'Raksha Bandhan',
       'Rosh Hashanah', 'Lailat al Qadr', 'Christmas', 'Diwali',
       'Birth of Guru Nanak Dev Sahib Ji', 'Ash Wednesday',
       'Mahayana New Year', 'Nirvana Day', 'Martyrdom of the Bab',
       'Eid ul Fitr', 'Ashala Puja / Dharma Day',
       "Ascension of Baha'ullah", 'Hijra', "Birth of Baha'ullah",
       'Birthday of Guru Gobind Singh Ji', 'Epiphany ',
       'Chinese New Year', 'Theravada New Year', 'Holi', 'Shavuot',
       'Assumption of the Blessed Virgin Mary', 'Bodhi Day',
       'Tet Nam Moiaabs', 'New Ruz', 'Good/Holy Friday',
       'Easter/Pascha [Orthodox]', 'Pentecost', 'Dussehra',
       'Bandi Chhor Divas (Diwali)', 'Birth of Bab', 'Yom

In [32]:
school_holidays = all_holidays_victoria_df[all_holidays_victoria_df["dateType"] == "SCHOOL_HOLIDAY"]


In [ ]:
# Convert string to datetime
school_holidays["important_date"] = pd.to_datetime(school_holidays["important_date"], dayfirst=True)  


/var/folders/83/z6dh6x592f7cw62tg8hz97000000gp/T/ipykernel_26076/653416732.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_holidays["important_date"] = pd.to_datetime(school_holidays["important_date"], dayfirst=True)


In [ ]:
import numpy as np
import re

# --- 0) Start from your dataframe of holidays only ---
hol = school_holidays.copy()
hol = hol[hol["dateType"].eq("SCHOOL_HOLIDAY")].copy()

# Parse the date column
hol["important_date"] = pd.to_datetime(hol["important_date"], dayfirst=True, errors="coerce")

# --- 1) Robust start/end detection (just the words 'start'/'end') ---
name_lower = hol["name"].str.lower()
hol["boundary"] = np.where(name_lower.str.contains(r"\bend\b"), "end",
                   np.where(name_lower.str.contains(r"\bstart\b"), "start", pd.NA))

# Keep only rows where we could tell start/end
hol = hol.dropna(subset=["boundary"])

# --- 2) Extract a clean key to pair starts with ends (e.g., 'Term 3 2025') ---
# Handles 'Term 3 2025', 'Term3 2025', extra spaces, etc.
hol["term"] = hol["name"].str.extract(r"term\s*([1-4])", flags=re.I, expand=True)
hol["year"] = hol["name"].str.extract(r"(20\d{2}|2030)", flags=re.I, expand=True)
hol = hol.dropna(subset=["term","year"])
hol["term_key"] = "Term " + hol["term"].astype(str) + " " + hol["year"].astype(str)

# --- 3) Get one start + end per term_key (min/max covers duplicates) ---
tidy = (hol.groupby(["term_key"])
          .agg(start=("important_date", lambda s: s[name_lower.loc[s.index].str.contains(r"\bstart\b")].min()),
               end  =("important_date", lambda s: s[name_lower.loc[s.index].str.contains(r"\bend\b")].max()))
          .reset_index())

# Drop incomplete or reversed intervals
tidy = tidy.dropna(subset=["start","end"])
tidy = tidy[tidy["start"] <= tidy["end"]].sort_values("start").reset_index(drop=True)

# --- 4) Weekly calendar with overlap logic (counts weeks that partially overlap) ---
calendar = pd.DataFrame({"week": pd.date_range("2014-01-01", "2030-12-31", freq="W-MON")})
calendar["week_end"] = calendar["week"] + pd.Timedelta(days=6)
calendar["is_holiday"] = 0

for _, r in tidy.iterrows():
    overlap = (calendar["week_end"] >= r["start"]) & (calendar["week"] <= r["end"])
    calendar.loc[overlap, "is_holiday"] = 1

holiday_weeks_victoria = calendar[["week", "is_holiday"]]

# Find the range of years actually present in your Excel file
min_year = hol["important_date"].dt.year.min()
max_year = hol["important_date"].dt.year.max()

# Keep only weeks within that year range
holiday_weeks_victoria = holiday_weeks_victoria[
    (holiday_weeks_victoria["week"].dt.year >= min_year) &
    (holiday_weeks_victoria["week"].dt.year <= max_year)
].reset_index(drop=True)

holiday_weeks_victoria


,week,is_holiday
0,2019-01-07,0
1,2019-01-14,0
2,2019-01-21,0
3,2019-01-28,0
4,2019-02-04,0
...,...,...
360,2025-12-01,0
361,2025-12-08,0
362,2025-12-15,0
363,2025-12-22,0


In [39]:
school_holidays_all_states = pd.read_csv("school.holidays_all_states.txt")
school_holidays_all_states

,City,Date,schoolhols,school.hols
1,Adelaide,2004-01-01,1,4.0
2,Brisbane,2004-01-01,1,4.0
3,Canberra,2004-01-01,1,4.0
4,Darwin,2004-01-01,1,4.0
5,Hobart,2004-01-01,1,4.0
...,...,...,...,...
58436,Darwin,2023-12-31,1,4.0
58437,Hobart,2023-12-31,1,4.0
58438,Melbourne,2023-12-31,1,4.0
58439,Perth,2023-12-31,1,4.0


In [47]:
# Make sure Date column is datetime
school_holidays_all_states["Date"] = pd.to_datetime(school_holidays_all_states["Date"])

# Find min and max
min_date = school_holidays_all_states["Date"].min()
max_date = school_holidays_all_states["Date"].max()

print("Min date:", min_date)
print("Max date:", max_date)


Min date: 2004-01-01 00:00:00
Max date: 2023-12-31 00:00:00


In [83]:
# Concat both sources
combined = pd.concat([
    holiday_weeks_victoria.assign(source="vic"),
    mel_weekly[["week", "is_holiday"]].assign(source="mel")
])

# Prefer mel where overlap: sort so mel rows come first
combined = combined.sort_values(["week", "source"], ascending=[True, False])

# Drop duplicates, keeping the first occurrence (mel first)
combined = combined.drop_duplicates(subset="week", keep="first")

# Final combined dataframe
holiday_weeks_victoria = combined.sort_values("week")[["week", "is_holiday"]].reset_index(drop=True)

holiday_weeks_victoria


,week,is_holiday
0,2003-12-30,1
1,2004-01-06,1
2,2004-01-13,1
3,2004-01-20,1
4,2004-01-27,1
...,...,...
1404,2025-12-01,0
1405,2025-12-08,0
1406,2025-12-15,0
1407,2025-12-22,0


In [82]:
# Concat both sources
combined = pd.concat([
    holiday_weeks_victoria.assign(source="vic"),
    mel_weekly[["week", "is_holiday"]].assign(source="mel")
])

# Prefer vic where overlap: sort so vic rows come first
combined = combined.sort_values(["week", "source"], ascending=[True, True])

# Drop duplicates, keeping the first occurrence (vic first)
combined = combined.drop_duplicates(subset="week", keep="first")

# Final combined dataframe
holiday_weeks_victoria = combined.sort_values("week")[["week", "is_holiday"]].reset_index(drop=True)

holiday_weeks_victoria



,week,is_holiday
0,2003-12-30,1
1,2004-01-06,1
2,2004-01-13,1
3,2004-01-20,1
4,2004-01-27,1
...,...,...
1404,2025-12-01,0
1405,2025-12-08,0
1406,2025-12-15,0
1407,2025-12-22,0


In [65]:
# Filter Sydney only
syd = school_holidays_all_states[
    school_holidays_all_states["City"].str.lower() == "sydney"
].copy()

# 1) ensure datetime
syd["Date"] = pd.to_datetime(syd["Date"], errors="coerce")

# 2) map each day to its week start (Monday)
syd["week"] = syd["Date"].dt.to_period("W-MON").dt.start_time

# 3) weekly dummy: if any day in the week is a holiday -> 1
syd_weekly = (syd.groupby("week")["schoolhols"]
                .max()
                .reset_index()
                .rename(columns={"schoolhols": "is_holiday"}))

# optional: sort
syd_weekly = syd_weekly.sort_values("week").reset_index(drop=True)

syd_weekly

,week,is_holiday
0,2003-12-30,1
1,2004-01-06,1
2,2004-01-13,1
3,2004-01-20,1
4,2004-01-27,0
...,...,...
1039,2023-11-28,0
1040,2023-12-05,0
1041,2023-12-12,0
1042,2023-12-19,1


In [68]:
school_holidays_nsw = pd.read_csv("school_holidays_nsw.csv")
school_holidays_nsw

,START_DAY,END_DAY
0,29/01/2019,12/04/2019
1,29/04/2019,5/07/2019
2,22/07/2019,27/09/2019
3,14/10/2019,20/12/2019
4,28/01/2020,9/04/2020
5,27/04/2020,3/07/2020
6,20/07/2020,25/09/2020
7,12/10/2020,18/12/2020
8,27/01/2021,1/04/2021
9,19/04/2021,25/06/2021


In [70]:
import pandas as pd

nsw = school_holidays_nsw.copy()

# 1) Parse dates & sort
nsw["START_DAY"] = pd.to_datetime(nsw["START_DAY"], dayfirst=True, errors="coerce")
nsw["END_DAY"]   = pd.to_datetime(nsw["END_DAY"],   dayfirst=True, errors="coerce")
nsw = nsw.sort_values(["START_DAY", "END_DAY"]).reset_index(drop=True)

# 2) Holidays are the GAPS between consecutive terms:
#    holiday_start = previous term's END_DAY + 1
#    holiday_end   = next term's START_DAY - 1
nsw["next_start"]   = nsw["START_DAY"].shift(-1)
hol = pd.DataFrame({
    "holiday_start": nsw["END_DAY"]   + pd.Timedelta(days=1),
    "holiday_end":   nsw["next_start"] - pd.Timedelta(days=1),
})

# Keep valid gaps only
hol = hol.dropna()
hol = hol[hol["holiday_start"] <= hol["holiday_end"]].reset_index(drop=True)

# 3) Build weekly calendar over the covered range
weeks = pd.date_range(hol["holiday_start"].min().floor("D"),
                      hol["holiday_end"].max().ceil("D"),
                      freq="W-MON")
calendar = pd.DataFrame({"week": weeks})
calendar["week_end"] = calendar["week"] + pd.Timedelta(days=6)
calendar["is_holiday"] = 0

# 4) Mark weeks as holiday if the week interval overlaps any holiday interval
for _, r in hol.iterrows():
    overlap = (calendar["week_end"] >= r["holiday_start"]) & (calendar["week"] <= r["holiday_end"])
    calendar.loc[overlap, "is_holiday"] = 1

# 5) Final output in the same format as `syd_weekly`
nsw_weekly = calendar[["week", "is_holiday"]].sort_values("week").reset_index(drop=True)

nsw_weekly


,week,is_holiday
0,2019-04-15,1
1,2019-04-22,1
2,2019-04-29,0
3,2019-05-06,0
4,2019-05-13,0
...,...,...
334,2025-09-08,0
335,2025-09-15,0
336,2025-09-22,1
337,2025-09-29,1


In [ ]:
import pandas as pd
import numpy as np

# ensure same dtypes
syd_weekly["week"] = pd.to_datetime(syd_weekly["week"])
nsw_weekly["week"] = pd.to_datetime(nsw_weekly["week"])

combined = syd_weekly.merge(
    nsw_weekly, on="week", how="outer", suffixes=("_syd", "_nsw")
)

# use Sydney if it exists, otherwise NSW
combined["is_holiday"] = combined["is_holiday_syd"].where(
    combined["is_holiday_syd"].notna(), combined["is_holiday_nsw"]
).fillna(0).astype(int)

holiday_weeks_sydney = (combined[["week", "is_holiday"]]
                .sort_values("week")
                .reset_index(drop=True))
holiday_weeks_sydney



# School Holidays for Sydney and Melbourne

In [85]:
holiday_weeks_sydney

,week,is_holiday
0,2003-12-30,1
1,2004-01-06,1
2,2004-01-13,1
3,2004-01-20,1
4,2004-01-27,0
...,...,...
1378,2025-09-08,0
1379,2025-09-15,0
1380,2025-09-22,1
1381,2025-09-29,1


In [84]:
holiday_weeks_victoria

,week,is_holiday
0,2003-12-30,1
1,2004-01-06,1
2,2004-01-13,1
3,2004-01-20,1
4,2004-01-27,1
...,...,...
1404,2025-12-01,0
1405,2025-12-08,0
1406,2025-12-15,0
1407,2025-12-22,0
